In [175]:
# look at the TTS of various segments 
# date created: 3/3/2020
# author: sofia chelpon

In [1]:
############ --------------- PREP WORKSPACE --------------- ###########
import numpy as np 
import pandas as pd
import xarray as xr 

import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns

%matplotlib inline
%config inlinebackend.figure_format='retina'

# import my TTS module, need to add path to folder
import sys
sys.path.insert(1, '/Volumes/scdrive2/TTS_2020/base_tts_code/')
import tts_mod

In [2]:
############ --------------- LOAD SEGMENT DATA --------------- ###########
utbl_allseg = pd.read_pickle('/Volumes/scdrive2/TTS_2020/get_tts/tts_vary_ut/MD2_outputs/twp_awas_replace_082020_020N/utbl_allseg_campavgbl_tropotau_twp_awas_replace.pkl')
mustar_allseg = pd.read_pickle('/Volumes/scdrive2/TTS_2020/get_tts/tts_vary_ut/MD2_outputs/twp_awas_replace_082020_020N/mustar_allseg_campavgbl_tropotau_twp_awas_replace.pkl')
tau_allseg = pd.read_pickle('/Volumes/scdrive2/TTS_2020/get_tts/tts_vary_ut/MD2_outputs/twp_awas_replace_082020_020N/tau_allseg_campavgbl_tropotau_twp_awas_replace.pkl')
t_allseg = pd.read_pickle('/Volumes/scdrive2/TTS_2020/get_tts/tts_vary_ut/MD2_outputs/twp_awas_replace_082020_020N/t_allseg_campavgbl_tropotau_twp_awas_replace.pkl')
gf_allseg = pd.read_pickle('/Volumes/scdrive2/TTS_2020/get_tts/tts_vary_ut/MD2_outputs/twp_awas_replace_082020_020N/gf_allseg_campavgbl_tropotau_twp_awas_replace.pkl')
seg_info = pd.read_pickle('/Volumes/scdrive2/TTS_2020/get_tts/tts_vary_ut/MD2_outputs/twp_awas_replace_082020_020N/segment_info_campavgbl_tropotau_twp_awas_replace.pkl')
trcnames_allseg = pd.read_pickle("/Volumes/scdrive2/TTS_2020/get_tts/tts_vary_ut/MD2_outputs/twp_awas_replace_082020_020N/trcnames_allseg.pkl")

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/scdrive2/TTS_2020/get_tts/tts_vary_ut/MD2_outputs/twp_awas_replace_082020_020N/utbl_allseg_campavgbl_tropotau_twp_awas_replace.pkl'

In [ ]:
############ --------------- FILTER BY R^2 VALUES --------------- ###########
high_r2_idx = np.ndarray.flatten(np.argwhere(seg_info['r squared'].values >= 0.65))

seg_info_screened = seg_info.iloc[high_r2_idx]

# screen vars 
mustar_allseg_screened = mustar_allseg[high_r2_idx]
utbl_allseg_screened = utbl_allseg[high_r2_idx]
tau_allseg_screened = tau_allseg[high_r2_idx]
trcnames_allseg_screened = trcnames_allseg[high_r2_idx]

gf_allseg_screened = gf_allseg[high_r2_idx]
t_allseg_screened = t_allseg[high_r2_idx]

In [ ]:
############ --------------- FIND MIN/MAX CURVES --------------- ###########
min_mode_loc = np.argmin(seg_info_screened['mode age']) 
max_mode_loc = np.argmax(seg_info_screened['mode age']) 

min_mean_age = np.min(seg_info_screened['mean age']) 
max_mean_age = np.max(seg_info_screened['mean age']) 
print(min_mean_age, max_mean_age)

min_mode_age = np.min(seg_info_screened['mode age']) 
max_mode_age = np.max(seg_info_screened['mode age']) 
print(min_mode_age, max_mode_age)

# utbl  
min_utbl = utbl_allseg_screened[min_mode_loc].values
max_utbl = utbl_allseg_screened[max_mode_loc].values

min_r2 = np.min(seg_info_screened['r squared']) 
max_r2 = np.max(seg_info_screened['r squared']) 

min_mustar = mustar_allseg_screened[min_mode_loc].values
max_mustar = mustar_allseg_screened[max_mode_loc].values

min_tau = tau_allseg_screened[min_mode_loc].values
max_tau = tau_allseg_screened[max_mode_loc].values

min_gf = gf_allseg_screened[min_mode_loc].values
max_gf = gf_allseg_screened[max_mode_loc].values

min_t = t_allseg_screened[min_mode_loc].values
max_t = t_allseg_screened[max_mode_loc].values

print(seg_info_screened['mode age'].mean())
print(seg_info_screened['mean age'].mean())

In [ ]:
# look at the top 10, bottom 10 TTS sorted my mode 
ascend_mode = seg_info_screened.sort_values('mode age')
short10 = ascend_mode[:10]
long10 = ascend_mode[-10:]

In [ ]:
# get positions 
pos = pd.read_pickle('/Volumes/scdrive2/TTS_2020/get_tts/tts_vary_ut/data_prep/awas_positions.pkl')
pos = pos.drop('Notes', axis = 0).drop('Intrument', axis = 0)
pos_time = pos.iloc[0]
pos

In [ ]:
# add lat/lon to the top and bottom 10 dataframes 
# create columns 
fill = np.empty(10)
fill[:] = np.nan

short10['GGALT'] = fill 
short10['GGLAT'] = fill 
short10['GGLON'] = fill 

# loop through every segment, match the time to the position and add it 
for idx in (np.arange(0, len(short10))):
    seg = short10.iloc[idx]
    time = seg.Time_UTC
    where = np.argwhere(pos_time.values == time)
    info = pos.iloc[:,[np.ndarray.item(where)]]
    short10['GGALT'].iloc[idx] = np.ndarray.item(info.iloc[2].values)
    short10['GGLAT'].iloc[idx] = np.ndarray.item(info.iloc[3].values)
    short10['GGLON'].iloc[idx] = np.ndarray.item(info.iloc[4].values)
    
short10 = short10.reset_index()
short10 = short10.drop(short10.columns[0], axis=1)
short10

In [ ]:
# add lat/lon to the top and bottom 10 dataframes 
# create columns 
long10['GGALT'] = fill 
long10['GGLAT'] = fill 
long10['GGLON'] = fill 

# loop through every segment, match the time to the position and add it 
for idx in (np.arange(0, len(long10))):
    seg = long10.iloc[idx]
    time = seg.Time_UTC
    where = np.argwhere(pos_time.values == time)
    info = pos.iloc[:,[np.ndarray.item(where)]]
    long10['GGALT'].iloc[idx] = np.ndarray.item(info.iloc[2].values)
    long10['GGLAT'].iloc[idx] = np.ndarray.item(info.iloc[3].values)
    long10['GGLON'].iloc[idx] = np.ndarray.item(info.iloc[4].values)
    
long10 = long10.reset_index()
long10 = long10.drop(long10.columns[0], axis=1)
long10

In [ ]:
# plot prep 
mypath = '/Volumes/scdrive2/TTS_2020/contrast_readin/toga_lodhalf/toga_trace_gases_twp.nc'
toga_trc = xr.open_dataset(mypath)

In [ ]:
fig, ax = plt.subplots(figsize = [10, 10])
ax.scatter(long10.GGLON, long10.GGLAT, color = 'b', alpha = 0)
ax.scatter(short10.GGLON, short10.GGLAT, color = 'r', alpha = 0)
for i in np.arange(0, len(long10)):
    plt.text(long10.GGLON[i], long10.GGLAT[i], str(i), fontsize = 15, color = 'b')
    plt.text(short10.GGLON[i], short10.GGLAT[i], str(i), fontsize = 15, color = 'r')

ax.grid(which = 'major')
ax.set_title('Locations of Longest and Shortest Mode UT Segments', fontsize = 20)
ax.set_xlabel('Longitude', fontsize = 20)
ax.set_ylabel('Latitude', fontsize = 20)
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)

# landmarks
# guam 
plt.scatter(144.7, 13.4, s = 70, marker = 'x', color = 'k')
plt.text(145, 14, 'Guam', fontsize = 14)
# chuuk 
plt.scatter(151.7, 7.4, s = 70, marker = 'x', color = 'k')
plt.text(152, 7.4, 'Chuuk', fontsize = 14)
# palau 
plt.scatter(134.6, 7.5, s = 70, marker = 'x', color = 'k')
plt.text(135, 7.6, 'Palau', fontsize = 14)

# 
plt.plot(toga_trc.GGLON, toga_trc.GGLAT, color = 'k', alpha = 0.2)
